In [106]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import itertools


In [2]:
import pandas as pd
import pyarrow
import fastparquet

In [3]:
OB_train0 = pd.read_parquet('book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet', engine='pyarrow')
trade_train0 = pd.read_parquet('trade_train.parquet/stock_id=0/ef805fd82ff54fadb363094e3b122ab9.parquet', engine='pyarrow')
vol_train = pd.read_csv('train.csv')
vol_train0 = vol_train[vol_train['stock_id'] == 0]



In [7]:
OB_train0


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28


In [92]:
OB_train0_ = OB_train0[(OB_train0['time_id']>=5) & (OB_train0['time_id']<=100000)]
trade_train0_ = trade_train0[(trade_train0['time_id']>=5) & (trade_train0['time_id']<=100000)]

In [93]:
time_ids = (OB_train0_['time_id'].unique()).tolist()
# len(time_ids)


In [107]:
OB_train0

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28


In [108]:
trade_train0

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
...,...,...,...,...,...
123438,32767,471,0.998659,200,3
123439,32767,517,0.998515,90,1
123440,32767,523,0.998563,1,1
123441,32767,542,0.998803,90,4


In [94]:
full = pd.merge(OB_train0_,trade_train0_,how = 'inner',on = ['seconds_in_bucket','time_id'])   # this only gives us rows of seconds in a timebucket where a trade has occured. I have disregarded all times in the orderbook where no trade has occured


In [112]:
full[full['time_id']==]

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,price,size,order_count
40,11,0,0.999473,1.000176,0.999423,1.000376,205,100,20,30,0.999824,2,2
41,11,18,0.999473,1.000025,0.999423,1.000376,200,100,20,30,0.999925,1,1
42,11,28,0.999473,1.000326,0.999423,1.000376,200,100,20,30,0.999925,1,1
43,11,64,0.999473,1.000025,0.999423,1.000326,100,110,20,11,0.999724,1,1
44,11,75,0.999724,1.000376,0.999473,1.000527,1,30,100,150,1.000025,110,2
45,11,131,0.999573,1.000376,0.999473,1.000527,38,40,200,150,0.999925,1,1
46,11,164,0.999925,1.000226,0.999875,1.000376,1,10,20,31,0.999990,29,3
47,11,184,0.999724,1.000226,0.999573,1.000376,200,11,30,30,0.999875,20,1
48,11,202,0.999724,1.000126,0.999573,1.000226,312,10,30,100,0.999975,2,2
49,11,246,0.999724,1.000126,0.999674,1.000226,304,10,20,100,0.999724,3,1


In [95]:
# The df with the most rows is 117. I am going to pad all the other dfs with lots of zeros so that all the dfs have 117 rows.
# list of all dfs:

dfs = []
for t in time_ids:
    dfs.append(full[full['time_id'] == t])



In [96]:
padded_dfs = []

for df in dfs:
    if len(df)<117:
        zeros_df = pd.DataFrame(0,index = list(range(117-len(df))),columns=dfs[0].columns)
        p_df = pd.concat([zeros_df,df])
        padded_dfs.append(p_df)



In [97]:
# getting X, Y vals

# X = [full[full['time_id'] == i] for i in time_ids]
# y = [vol_train0[vol_train0['time_id'] == i]['target'].iloc[0] for i in time_ids]


In [102]:
X = padded_dfs
Y = [vol_train0[vol_train0['time_id'] == i]['target'].iloc[0] for i in time_ids]
Y = Y[0:len(padded_dfs)]

In [103]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [104]:
import numpy as np

# Flatten each 117x13 matrix into a 1D array
X_train_flat = [X.values.flatten() for X in X_train]
X_test_flat = [X.values.flatten() for X in X_test]

# Convert the lists to numpy arrays
X_train_flat = np.array(X_train_flat)
X_test_flat = np.array(X_test_flat)


In [105]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Training
rf_regressor.fit(X_train_flat, Y_train)

predictions = rf_regressor.predict(X_test_flat)

# Evaluate the model
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 8.198984190807856e-06
